<a href="https://colab.research.google.com/github/FelipeTufaile/MixtureModels/blob/main/MixtureModels/tests/Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Circle, Arc

In [66]:
class GaussianMixture():
  """
  Tuple holding a gaussian mixture

  mu: np.ndarray  # (K, d) array - each row corresponds to a gaussian component mean
  var: np.ndarray  # (K, ) array - each row corresponds to the variance of a component
  p: np.ndarray  # (K, ) array = each row corresponds to the weight of a component
  """

  def __init__(self, k, seed = 0):
    """
    Initializes the gaussian mixture model.

    Args:
        K (int): number of components
        seed (int): random seed

    Returns:
        mixture: the initialized gaussian mixture model
    """
    # Storing k
    self.k = k

    # Storing seed
    self.seed = seed

    # Initializing the weight (mixing proportions) for each component (cluster)
    self.p = np.ones(k) / k


  def fit(self, X):
    """
    Initializes the mixture model with random points as initial means and uniform assingments

    Args:
        X: (n, d) array holding the data

    Returns:
        mixture: the initialized gaussian mixture
        post: (n, K) array holding the soft counts for all components for all examples
    """
    # Setting seed
    np.random.seed(self.seed)

    # Calculating the number of samples "n" in the dataset (X array)
    n, d = X.shape

    # Initialize the mean array with random points from X as initial means
    self.mu = X[np.random.choice(n, self.k, replace=False)]

    # Initialize the variance array as an array of zeros
    self.var = np.zeros((self.k, d))

    # Update the variance array for each component (cluster)
    for j in range(self.k):
        self.var[j,:] = np.mean((X-self.mu[j,:])**2, axis=0)

    # Calculating posterior
    post = np.ones((n, self.k)) / self.k

    return self

In [40]:
# Generating a random Gaussian distribution
samples1 = np.random.normal(np.array([5.0, 5.0]), np.array([0.25, 0.25]), (100,2))
samples2 = np.random.normal(np.array([7.5, 7.5]), np.array([0.50, 0.50]), (100,2))
samples3 = np.random.normal(np.array([2.5, 2.5]), np.array([0.75, 0.75]), (100,2))

# Concatenate the arrays vertically
X = np.concatenate((samples1, samples2, samples3))

In [67]:
# Initializing a Gaussian Mixture Model
GMM = GaussianMixture(k=3)

In [68]:
# Fitting a sample distribution to the model
GMM = GMM.fit(X=X)


In [69]:
# Printing Gaussian Mixture Model parameters
for i, params in enumerate(zip(GMM.mu, GMM.var)):
  print(f"Center of distributions {str(i).zfill(3)} is {params[0]} and variance is {params[1]}")



Center of distributions 000 is [2.52775429 3.07592681] and variance is [10.41441052  8.05249211]
Center of distributions 001 is [8.11243528 7.53242805] and variance is [14.6654972  11.30565657]
Center of distributions 002 is [4.91150152 4.65626218] and variance is [4.59871367 4.66080138]


In [ ]:
def GaussianProbability(X, mu, var):
    """
    The function calculates the probability of X belonging to the Gaussian distribution using the multivariate Gaussian PDF formula.

    Args:
        X (numpy.ndarray): Point with shape (1, d).
        mu (numpy.ndarray): Mean of the Gaussian distribution with shape (1, d).
        var (numpy.ndarray): Variance of the Gaussian distribution with shape (1, d).

    Returns:
        float: Probability of X belonging to the Gaussian distribution.
    """
    # Ajusting the shape of X
    X = X.reshape(1,-1)

    # Ajusting the shape of mu
    mu = mu.reshape(1,-1)

    # Ajusting the shape of var
    var = var.reshape(1,-1)

    # Calculate the probability from the Gaussian Probability Density Function (PDF)
    probability = np.exp(-0.5 * np.sum((X - mu)**2 / var)) / ((2 * np.pi)**(X.shape[1] / 2) * np.prod(np.sqrt(var)))

    return probability


In [98]:
from os import killpg
def estep(X, GMM, GaussianProbability=GaussianProbability):
    """
    E-step: Softly assigns each datapoint to a gaussian component

    Args:
        X: (n, d) array holding the data mixture: the current gaussian mixture

    Returns:
        np.ndarray: (n, K) array holding the soft counts for all components for all examples
        float: log-likelihood of the assignment
    """

    # Initializing the soft counts (posterior)
    post = []

    # Initializing the log likelihood parameter as zero
    l_theta = np.zeros((1,GMM.k))

    # Iterate over all datapoints
    for i in range(0, X.shape[0]):

        # Calculating the likelihood for the datapoint [i] belonging to each of the clusters [j]
        p_x_theta = np.array([GMM.p[j]*GaussianProbability(X=X[i,:], mu=GMM.mu[j,:], var=GMM.var[j,:]) for j in range(GMM.k)])

        # Calculating the likelihood for each cluster
        post.append(p_x_theta/np.sum(p_x_theta))

        # Updating the log-likelihood
        l_theta += np.log(p_x_theta)

    return np.array(post), l_theta

In [100]:
post, l_theta = estep(X=X, GMM=GMM)

In [ ]:
def mstep(X, post, GMM):
    """
    M-step: Updates the gaussian mixture by maximizing the log-likelihood of the weighted dataset

    Args:
        X: (n, d) array holding the data
        post: (n, K) array holding the soft counts for all components for all examples

    Returns:
        GaussianMixture: the new gaussian mixture
    """
    # Updating n_hat
    n_hat = np.sum(post, axis=0)

    # Updating the mixture proportions
    p_hat = n_hat/X.shape[0]

    GMM.mu = (X.T@post/np.sum(post, axis=0))

    GMM.var = np.sum(post @ np.array([np.sum((GMM.mu[j] - X)**2, axis=0) for j in range(GMM.k)]), axis = 0)

    for j in range(GMM.k):
        mu[j, :] = post[:, j].T @ X / n_hat[j]
        sse = ((mu[j] - X)**2).sum(axis=1) @ post[:, j]
        var[j] = sse / (d * n_hat[j])

    return GaussianMixture(mu, var, p)


In [71]:
(1,300) (300, 2)

array([ 89.08231585,  75.69866889, 135.21901527])

In [107]:
post

array([[0.20102874, 0.1311185 , 0.66785276],
       [0.21906649, 0.11906602, 0.66186749],
       [0.1922101 , 0.14824839, 0.65954151],
       [0.19232274, 0.14052888, 0.66714838],
       [0.20874122, 0.13055181, 0.66070697],
       [0.2281924 , 0.11117516, 0.66063244],
       [0.18828247, 0.14519792, 0.66651961],
       [0.19067043, 0.14264908, 0.66668049],
       [0.1995996 , 0.13255325, 0.66784716],
       [0.20785412, 0.13025721, 0.66188867],
       [0.21736305, 0.11924076, 0.66339619],
       [0.20700498, 0.12803931, 0.66495571],
       [0.21891829, 0.11681522, 0.6642665 ],
       [0.23116314, 0.10862798, 0.66020887],
       [0.21086147, 0.12247009, 0.66666843],
       [0.2140888 , 0.12371115, 0.66220005],
       [0.19768346, 0.14171898, 0.66059756],
       [0.21444798, 0.12443455, 0.66111747],
       [0.21343656, 0.12076599, 0.66579746],
       [0.19854986, 0.13413813, 0.66731201],
       [0.18799873, 0.14669637, 0.6653049 ],
       [0.17239799, 0.16772126, 0.65988075],
       [0.